In [1]:
%run "./preprocessing.ipynb"

Done !!


In [2]:
import warnings
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import Dense, Input, LSTM, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, SGD
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

tf.__version__
warnings.filterwarnings(action='ignore')

In [3]:
e_commerce_fc_data.head(1)

,창고코드,주문 날짜,주문 시간,고객사코드,품목수량,입력자ID,권역구분,송화인주소1,송화인주소2,수화인주소1,...,datetime,송화인터미널,수화인터미널,SHPR_ADDR_LATITUDE,SHPR_ADDR_LONGITUDE,SHPR_ADDR_HUBS,CNEE_ADDR_LATITUDE,CNEE_ADDR_LONGITUDE,CNEE_ADDR_HUBS,distance
2021-03-03 03:55:21,KX007,20210303,35521,90001441,1,iNEXBatch,50.0,경기도,광주시,경상남도,...,2021-03-03 03:55:21,경기도 광주시,경상남도 김해시,37.403,127.239,분당ASub,35.236,128.831,김해ASub,340.199185


In [4]:
np.random.seed(seed=42)    
random_number = np.random.randint(0, len(e_commerce_fc_data), 150000)

selected_data = e_commerce_fc_data.iloc[random_number, :][["datetime", "고객사코드", "distance"]]

In [10]:
### 출고량 별 세분화 
def set_fee(selected):
    code_fee_ = {
        100000 : 0,
        50000 : 20,
        30000 : 40,
        20000 : 60,
        10000 : 80,
        5000 : 100,
        3000 : 120,
        2000 : 140,
        1000 : 160,
        500 : 180,
        0 : 200
    }

    code_list = {
        90001302 : 100000,
        90001542 : 100000,
        90001341 : 100000,
        90001541 : 100000,
        90001441 : 100000,
        90001602 : 100000,
        90001443 : 100000,
        90001582 : 81950,
        90001622 : 68074,
        90001521 : 58814,
        90001682 : 36071,
        90001702 : 30606,
        90001776 : 24318,
        90001705 : 23976,
        90001704 : 20317,
        90001768 : 12844,
        90001662 : 8126,
        90001683 : 5850,
        90001381 : 5438,
        90001703 : 2416,
        90001664 : 1904,
        90001502 : 133,
        90001765 : 48,
        90001842 : 44,
        90001562 : 10,
        90001774 : 7,
        90001802 : 6
    }

    ### 거리별  세분화
    distance_fee_ = {
        400 : 500,
        300 : 400,
        100 : 300,
        50 : 200,
        0 : 0
    }

    ### 시간별 세분화
    time_fee_ = {
        400 : [1, 2, 11, 12, 13],
        200 : [0, 3, 4, 10, 14],
        100 : [5, 6, 7, 8, 9, 15, 23],
        0 : [16, 17, 18, 19, 20, 21, 22]
    }

    str_target_code = []
    make_code_fee_ = []
    make_distance_fee_ = []
    make_time_fee_ = []
    make_fee_ = []
    code_values = []
    code_fee_key = list(code_fee_.keys())
    distance_fee_key = list(distance_fee_.keys())
    time_fee_key = list(time_fee_.keys())

    for i in range(len(selected)):
        target_code = selected["고객사코드"][i]
        target_distance = selected["distance"][i]
        target_day = selected["datetime"][i]
        str_target_code.append(str(target_code))

        ## code fee_ 
        code_value = code_list[target_code]

        if code_value == 100000:
            set_code_fee = 0
            make_code_fee_.append(set_code_fee)
            code_values.append(code_value)
        else:
            for i in range(len(code_fee_key) -1 ):

                if (code_fee_key[i] > code_value) and ( code_fee_key[i+1] <= code_value) :
                    set_code_fee = code_fee_[code_fee_key[i+1]]
                    set_code_fee = set_code_fee + (code_fee_[code_fee_key[i+1]] - code_fee_[code_fee_key[i]]) * ( (code_value - code_fee_key[i+1]) / (code_fee_key[i] - code_fee_key[i+1]) )

                    if set_code_fee > 200:
                        set_code_fee = 200

                    make_code_fee_.append(int(set_code_fee))
                    code_values.append(code_value)

                else:
                    pass

        ## distance fee_
        if target_distance >= distance_fee_key[0]:
            set_distance_fee = 500
            make_distance_fee_.append(set_distance_fee)
        else:
            for i in range(len(distance_fee_key) -1 ):
                if (distance_fee_key[i] > target_distance) and ( distance_fee_key[i+1] <= target_distance):
                    set_distance_fee = distance_fee_[distance_fee_key[i+1]]
                    set_distance_fee = set_distance_fee + (distance_fee_[distance_fee_key[i+1]] - distance_fee_[distance_fee_key[i]]) * (( target_distance - distance_fee_key[i+1]) / ( distance_fee_key[i] - distance_fee_key[i+1]) )

                    if set_distance_fee > 500:
                        set_distance_fee = 200

                    if set_distance_fee < 0:
                        set_distance_fee = 0
                        
                    make_distance_fee_.append(int(set_distance_fee))
                else:
                    pass

        ## day fee_

        day_value = target_day.hour

        for i in time_fee_key:
            if day_value in time_fee_[i]:
                set_day_fee = i
                make_time_fee_.append(set_day_fee)            
            else:
                pass

    print(len(make_code_fee_))
    print(len(make_distance_fee_))
    print(len(make_time_fee_))    
    for i in range(len(selected)):
        make_fee_.append(
            2100 
            + make_code_fee_[i]
            + make_distance_fee_[i]
            + make_time_fee_[i]
        )


    selected["result"] = make_fee_
    selected["code"] = make_code_fee_
    selected["str_code"] = str_target_code
    selected["code_value"] = code_values
    return selected

In [11]:
set_fee_df = set_fee(selected_data)

150000
150000
150000


In [7]:
set_fee_df

,datetime,고객사코드,distance,result,code,str_code
2021-03-15 21:47:56,2021-03-15 21:47:56,90001302,70.990083,2258,0,90001302
2021-05-07 05:39:21,2021-05-07 05:39:21,90001541,63.136312,2373,0,90001541
2021-03-16 18:27:14,2021-03-16 18:27:14,90001541,268.493797,2315,0,90001541
2021-06-16 07:27:41,2021-06-16 07:27:41,90001768,91.661964,2401,85,90001768
2021-03-28 14:09:05,2021-03-28 14:09:05,90001541,62.113800,2475,0,90001541
...,...,...,...,...,...,...
2021-03-14 03:51:56,2021-03-14 03:51:56,90001441,60.870158,2478,0,90001441
2021-03-08 16:10:52,2021-03-08 16:10:52,90001521,254.217086,2345,23,90001521
2021-03-23 06:54:53,2021-03-23 06:54:53,90001441,252.141436,2423,0,90001441
2021-04-11 09:10:58,2021-04-11 09:10:58,90001341,53.283066,2393,0,90001341


In [ ]:
set_hour = []
for i in range(len(set_fee_df)):

    set_hour.append(set_fee_df["datetime"][i].hour)

set_fee_df["time"] = set_hour

In [ ]:
train_x = set_fee_df[["distance", "time", "str_code"]].iloc[:120000, :]
train_y = set_fee_df[["result"]].iloc[:120000, :]

test_x = set_fee_df[["distance", "time", "str_code"]].iloc[120000:, :]
test_y = set_fee_df[["result"]].iloc[120000:, :]

In [ ]:
svr = SVR()
rfg = RandomForestRegressor()
abr = AdaBoostRegressor()
dtr = DecisionTreeRegressor()

In [ ]:
svr.fit(train_x, train_y)
svr_pred = svr.predict(test_x)
print("mae : ", mean_absolute_error(svr_pred, test_y), "mse : ", mean_squared_error(svr_pred, test_y))

In [ ]:
rfg.fit(train_x, train_y)
rfg_pred = rfg.predict(test_x)
print("mae : ", mean_absolute_error(rfg_pred, test_y), "mse : ", mean_squared_error(rfg_pred, test_y))

In [ ]:
abr.fit(train_x, train_y)
abr_pred = abr.predict(test_x)
print("mae : ", mean_absolute_error(abr_pred, test_y), "mse : ", mean_squared_error(abr_pred, test_y))

In [ ]:
dtr.fit(train_x, train_y)
dtr_pred = dtr.predict(test_x)
print("mae : ", mean_absolute_error(dtr_pred, test_y), "mse : ", mean_squared_error(dtr_pred, test_y))

In [ ]:
batch_size = 100
model = Sequential()
model.add(Dense(units=64, activation="relu", input_shape=(batch_size, 3, )))
model.add(Dense(32))
model.add(Dense(1))

model.compile(loss="mean_squared_error", optimizer="adam")

model.fit(train_x, train_y, batch_size = batch_size, epochs=2000, verbose= 2)

In [ ]:
pred = model.predict(test_x)

In [ ]:
mean_absolute_error(pred, test_y), mean_squared_error(pred, test_y)

In [ ]:
with open("./model/set_fee_mode.pickle", "wb") as f:
    pickle.dump(rfg, f, pickle.HIGHEST_PROTOCOL)
    

import time

a = 10
while True:
    time.sleep(10)

    print(a)
    a= a + 1